In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_113917/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
# analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_10_9_0,0.998182,0.997845,0.999835,0.998973,0.219278,0.253156,0.028084,0.147239,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
1,model_10_9_1,0.998171,0.996285,0.999661,0.998199,0.220567,0.436449,0.057727,0.258228,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
2,model_10_8_13,0.998145,0.986739,0.996775,0.993117,0.223711,0.899240,0.428568,0.677748,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
3,model_10_8_12,0.998144,0.986769,0.996977,0.993256,0.223780,0.897257,0.401772,0.664089,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
4,model_10_8_14,0.998141,0.986712,0.996587,0.992988,0.224167,0.901124,0.453536,0.690494,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
1144,model_7_6_6,0.960579,0.970587,0.975799,0.978452,4.754183,1.786281,6.935601,4.209493,"Hidden Size=[25, 15], regularizer=0.01, learni..."
1160,model_4_7_1,0.958926,0.991601,0.991143,0.991412,4.953552,1.217157,1.046877,1.137025,"Hidden Size=[25], regularizer=0.1, learning_ra..."
1165,model_1_4_8,0.958514,0.949219,0.978687,0.967741,5.003167,4.698500,3.672296,4.215579,"Hidden Size=[30], regularizer=0.01, learning_r..."
1192,model_8_5_2,0.956034,0.947876,0.965830,0.964708,5.302346,4.545814,5.855629,5.162197,"Hidden Size=[25, 15], regularizer=0.1, learnin..."
